In [1]:
import pymongo
import pandas as pd
#from pandas import Series
import numpy as np
np.set_printoptions(threshold='nan')
from numpy.linalg import inv
from scipy import signal
from scipy.io import wavfile
import soundfile as sf
import matplotlib.pyplot as plt
%matplotlib inline
#from collections import Counter
from bson.objectid import ObjectId
import json
import pprint
from pydub import AudioSegment
import urllib
import fnmatch
import os
import requests
import ffmpy
import subprocess
import librosa
import librosa.display

In [2]:
client = pymongo.MongoClient('localhost',27017)
#client = pymongo.MongoClient('localhost',32768)
db = client.get_database('b=bandhub')

In [3]:
songCol = db.get_collection('songsStream')
vidCol = db.get_collection('mergedVideos')
trackCol = db.get_collection('tracksStream')
postCol = db.get_collection('posts')
[songCol.count(), vidCol.count(), trackCol.count(), postCol.count()]

[425706, 198169, 915582, 494867]

Find mixes with "NumTrack" published tracks, where NumTrack > 3

In [4]:
NumTrackMixes = []
masterSongIDs = []
NumTrack = 5
#zeroTrackMixes = 0
pubTracks = postCol.find({'songAccess':{'$exists': True}}, {'participantsInfo':1 ,'songAccess':1, 'objectId':1})
for docs in pubTracks:
    if(docs['songAccess'] == 1):
        if(len(docs['participantsInfo']['publishedTracks']) == NumTrack):
            #ThreeTrackMixes.append(docs['objectId'])
            NumTrackMixes.append(docs['_id'])
            masterSongIDs.append(docs['objectId'])
countNumTrack = len(NumTrackMixes)
#print(countNumTrack)

Storing the indices of the mixes with "NumTrack" published tracks, which contain multiple mixing coefficients

In [5]:
index_required = []
missing_SongID = []
missing_pubtrack = []
for j in range(countNumTrack):
    remix_test = NumTrackMixes[j]
    songID = masterSongIDs[j]
    
    pub_t = []
    rmt1 = postCol.find({'_id': remix_test})
    for docs in rmt1:
        for i in range(NumTrack):
            pub_t.append(docs['participantsInfo']['publishedTracks'][i]['_id'])
    
    songLookUp = songCol.find({'masterSongId' : songID})
    if songLookUp is -1:
        pprint.pprint('Error: SongID not found in Song Collection!')
        missing_SongID.append(songID)
    else:
        for songdocs in songLookUp:
            for i in range(NumTrack):
                if(str(pub_t[i]) not in songdocs['settings']):
                    missing_pubtrack.append(pub_t[i])   
                    #break
                else:
                    if ('volume' in songdocs['settings'][str(pub_t[i])]['audioChannels'][0] and 'volume' in songdocs['settings'][str(pub_t[i])]):
                        if (songdocs['settings'][str(pub_t[i])]['audioChannels'][0]['volume'] != songdocs['settings'][str(pub_t[i])]['volume']):
                            index_required.append(j)
                            #break

KeyError: 'audioChannels'

In [ ]:
print(index_required)

In [ ]:
print(missing_pubtrack)

In [ ]:
def get_filename(file):
    filePath = file.split('/')
    return filePath[- 1]

Storing the published tracks of each mix using the stored indices 

In [37]:
#for k in range(len(index_required)):
    #idx = index_required[k]
idx = 29
SongID = masterSongIDs[idx]
postID = NumTrackMixes[idx]
pub_tracks = []
pub_t_vol = []
pub_t_vol_ch = []
ogg_paths = []
start_times = []
track_filenames = []
songDoc = songCol.find({'masterSongId' : SongID})
rmtDoc = postCol.find({'_id': postID})

for rdocs in rmtDoc:
    for i in range(NumTrack):
        pub_tracks.append(rdocs['participantsInfo']['publishedTracks'][i]['_id'])
        
for sdocs in songDoc:
    for i in range(len(pub_tracks)):
        pub_t_vol.append(sdocs['settings'][str(pub_tracks[i])]['volume'])
        pub_t_vol_ch.append(sdocs['settings'][str(pub_tracks[i])]['audioChannels'][0]['volume'])
        gettrack = trackCol.find({'_id' : pub_tracks[i]})
        for tdocs in gettrack:
            ogg_paths.append(tdocs['audioChannels'][0]['fileUrl'])
            start_times.append(tdocs['startTimeValue'])
        for file in ogg_paths:
            get_filename(file)
    
pprint.pprint(pub_t_vol)
pprint.pprint('============')
pprint.pprint(pub_t_vol_ch)
pprint.pprint('============')
pprint.pprint(track_filenames)
pprint.pprint('============')
pprint.pprint(start_times)

KeyError: 'volume'

In [ ]:
def get_filename(file):
    filePath = file.split('/')
    return filePath[- 1]

In [ ]:
def download_track(ogg_path):
    
    path = '/scratch/rrs432/tracks'
    #os.mkdir(path)
    os.chdir(path)
    

    r = requests.get(ogg_path)
    filename = get_filename(ogg_path)
    with open(filename, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=128):
            fd.write(chunk)
            
    print('Track download successful!')

In [6]:
def clear_downloads():
    ! rm -rf *.ogg *.wav

In [7]:
clear_downloads()

In [ ]:
for i in range(NumTrack):
    download_track(ogg_paths[i])
    
pointer = vidCol.find({'songId': SongID})
for songdocs in pointer:
    premix = songdocs['oggMergedVideoUrl']

download_track(premix)

print('BandHub Mix Video downloaded successfully')
print('============')
!ls
print('============')